Write a program to estimate a depth map from a pair of calibrated images in Figure 4 using an MRF and graphcuts. The camera matrices are available here. Note that, for finding the epipolar lines using the provided camera matrices, you might want to use the following equation:

### camera metrices:
1221.2270770	0.0000000	479.5000000	
0.0000000	1221.2270770	269.5000000	
0.0000000	0.0000000	1.0000000	
1.0000000000	0.0000000000	0.0000000000	
0.0000000000	1.0000000000	0.0000000000	
0.0000000000	0.0000000000	1.0000000000	
0.0000000000	0.0000000000	0.0000000000	


1221.2270770	0.0000000	479.5000000	
0.0000000	1221.2270770	269.5000000	
0.0000000	0.0000000	1.0000000	
0.9998813487	0.0148994942	0.0039106989	
-0.0148907594	0.9998865876	-0.0022532664	
-0.0039438279	0.0021947658	0.9999898146	
-9.9909793759	0.2451742154	0.1650832670	

In [94]:
import numpy
import cv2
from gco import pygco
import matplotlib.image
import matplotlib.pyplot

In [95]:
# @cache
def compute_distance(point1: numpy.ndarray, point2: numpy.ndarray) -> float:
    """
    compute distance of two pixels
    Parameters
    ----------
    point1: ndarray, uint8, shape=(3)
        point one
    point2: ndarray, uint8, shape=(3)
        point two
    """
    # return (abs(point1[0]-point2[0]) + abs(point1[1] - point2[1]) + abs(point1[2] - point2[2]) ) / 3 / 255
    # if not dividing 255 as the C pseudocode example, there will be a overflowing of MAX_VALUE of Integer when calling pycgo.cut_grid_graph
    return numpy.sum(numpy.abs(point1-point2)) /3 / 255

In [96]:

K1: numpy.mat = numpy.mat(data=[[1221.2270770,0,479.5],[0,1221.2270770,269.5],[0,0,1]])
R1: numpy.mat = numpy.mat(data=[[1,0,0],[0,1,0],[0,0,1]])
T1: numpy.mat = numpy.mat(data=[0,0,0]).T
K2: numpy.mat = numpy.mat(data=[[1221.2270770,0,479.5],[0,1221.227077,269.5],[0,0,1]])
R2: numpy.mat = numpy.mat(data=[[0.9998813487,0.0148994942,0.0039106989],[-0.0148907594,0.9998865876,-0.0022532664],[-0.0039438279,0.0021947658,0.9999898146]])
T2: numpy.mat = numpy.mat(data=[-9.9909793759,0.2451742154,0.1650832670]).T
K1_inv: numpy.mat =  numpy.mat(data=numpy.linalg.inv(K1))


In [97]:

img1: numpy.ndarray = cv2.imread("data/not_rectified_image_1.jpeg", cv2.IMREAD_COLOR)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
height1, width1, _ = img1.shape

img2: numpy.ndarray = cv2.imread("data/not_rectified_image_2.jpeg", cv2.IMREAD_COLOR)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2 = cv2.resize(img2, (width1, height1))

height2, width2, _ = img2.shape

# img1.reshape(1,-1,3)
DEPTH_RANGE:int = 50
unary_cost = numpy.ndarray = numpy.zeros(shape=(height1, width1, DEPTH_RANGE), dtype=numpy.float64)
    # part1 = K2.dot(R2.T).dot(R1).dot(invK1).dot(Xh)

point_part_matrice = K2 @ R2.T @ R1 @ K1_inv
print(f"point_part_matrice:{point_part_matrice}")
epipolar_line_matrice = K2 @ R2.T @ (T1-T2)
for d in range(DEPTH_RANGE):
    epipolar_line = d * epipolar_line_matrice
    for y1 in range(height1):
        for x1 in range(width1):
            xh1 = numpy.mat(data=[y1,x1, 1])
            xh2 = (point_part_matrice @ xh1.T) + epipolar_line
            y2, x2, scale = xh2
            y2 = int(y2/scale)
            x2 = int(x2/scale)
            # print(f"y2:{y2},x2:{x2},scale:{scale}")
            if y2 < 0 or y2 >= height2 or x2 < 0 or x2 >= width2:
                unary_cost[y1, x1, d] = 1
            else:
                unary_cost[y1, x1, d] = compute_distance(point1=img1[y1,x1], point2=img2[y2, x2])
print(f"unary_cost:{unary_cost}")
i = numpy.arange(DEPTH_RANGE, dtype=numpy.int32)
j = numpy.arange(DEPTH_RANGE, dtype=numpy.int32)
fp = numpy.abs(i[: , numpy.newaxis] - j)
# LAMBDA = 0.01
lam = 0.01
while lam <= 1:
    labels = pygco.cut_grid_graph_simple(unary_cost=unary_cost, pairwise_cost=fp * lam, connect=8, algorithm='swap')
    labels = labels.reshape(unary_cost.shape[0], unary_cost.shape[1])
    print(f"labels:{labels}")
    img_path = f"data/no-rectified-labels{lam}.png"
    cv2.imwrite(img_path, labels)
    img_show = matplotlib.image.imread(img_path)
    matplotlib.pyplot.title(img_path)
    matplotlib.pyplot.imshow(X=img_show, cmap='gray') 
    matplotlib.pyplot.show()
    lam += 0.01



                

            


    



point_part_matrice:[[ 1.00141684e+00 -1.57754771e-02 -1.24907567e+00]
 [ 1.57625060e-02  9.99389337e-01 -4.71598561e+00]
 [ 3.20227006e-06 -1.84508389e-06  9.98951576e-01]]
